In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.2`

import $ivy.$                                  

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}


# Start

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Hello Spark")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/10/12 14:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/12 14:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/10/12 14:13:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


spark: SparkSession = org.apache.spark.sql.SparkSession@6f1947ac
import spark.implicits._

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.1.2


## DataSet

In [6]:
case class Person(name: String, age: Long)

val caseClassDS = Seq(Person("Andy", 32), Person("Mike", 25), Person("Mary", 18)).toDS()

defined class Person
caseClassDS: Dataset[Person] = [name: string, age: bigint]

In [7]:
caseClassDS.show

+----+---+
|name|age|
+----+---+
|Andy| 32|
|Mike| 25|
|Mary| 18|
+----+---+



In [8]:
caseClassDS.filter(_.age > 20).show

+----+---+
|name|age|
+----+---+
|Andy| 32|
|Mike| 25|
+----+---+



In [9]:
caseClassDS.select(col("name")).show

+----+
|name|
+----+
|Andy|
|Mike|
|Mary|
+----+

